# Assignment 3

In [248]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from numpy import pi, cos, sin
from scipy.signal import correlate, square

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [80]:
# Import data
D = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=1, usecols=[0,35])

In [81]:
# Convert time in D from MatLab time to Python Time
D['Time'] = D['NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
D2 = D.rename(index=str, 
                columns={"NaN": "MatLab Time", "48.4623": "Temperature"})

In [82]:
# Reorder columns 
cols = D2.columns.tolist()
cols = cols[-1:] + cols[:-1]
D23 = D2[cols]

# Set time as index column
DH = D23.set_index('Time')

In [133]:
# Select the dates:
# Hour resolution data
DH1 = DH.loc['2015-12-01 00:00':'2016-03-01 23:00']['Temperature']
DH2 = DH.loc['2016-06-01 00:00':'2016-09-01 23:00']['Temperature']

## Question 2: Quality of dummy weather forcast

### 1) Winter data set:

In [151]:
plt.figure(figsize = (15, 6))
ax1 = DH1.plot(label = 'Temperature')
plt.title('Temperature data from UVic Sci from 1 Dec 2015 to 1 Mar 2016', fontsize=16)
plt.xlabel('Time (hr)', fontsize=14)
plt.ylabel('Temperature ($^\circ C$)', fontsize=14)
ax1.xaxis.set_major_locator(mp.dates.DayLocator(bymonthday = (1, 8, 15, 22)))
plt.show()

<Figure size 1080x432 with 1 Axes>

#### Correlation

In [142]:
correlate(DH1, DH1)

array([ 42.9226,  82.1312, 122.9707, ..., 122.9707,  82.1312,  42.9226])

### 2) Summer data set:

In [150]:
plt.figure(figsize = (15, 6))
ax2 = DH2.plot(label = 'Temperature')
plt.title('Temperature data from UVic Sci from 1 Jun 2016 to 1 Sep 2016',  fontsize=16)
plt.xlabel('Time (hr)', fontsize=14)
plt.ylabel('Temperature ($^\circ C$)', fontsize=14)
ax2.xaxis.set_major_locator(mp.dates.DayLocator(bymonthday = (1, 8, 15, 22)))
plt.show()

<Figure size 1080x432 with 1 Axes>

# Question 3: Fourier series of a square wave

$$x(t) = 
\begin{cases} 
  -1 & -\frac{T}{2} < t < 0 \\
  1 & 0 \leq t < \frac{T}{2} 
\end{cases} 
\ \ \ \ \ \ \ \ \
x(t+nT) = x(t)$$

We found that:

$$x(t) \sim \frac{1}{2} \sum_{m=-\infty}^{\infty} S_m e^{i\left(\frac{m\pi}{T} \right) t}$$

$$S_m = \frac{i}{m\pi} \left[ 1 - \cos\left(\frac{m\pi}{2}\right)\right] =
\begin{cases} 
  \frac{2i}{m\pi} & \text{for } m \text{ odd} \\
  0 & \text{for } m \text{ even}
\end{cases}$$

We can reduce this into a Fourier sine series, which makes calulations easier:

$$x(t) \sim 2\sum_{n=1}^{\infty} b_m \sin\left(\frac{m\pi}{T} t \right)$$

$$b_m = \frac{1}{m\pi} \left[ 1 - \cos\left(m\pi\right)\right] =\frac{1}{m\pi} \left[ 1 - (-1)^m\right]
\begin{cases} 
  \frac{2}{m\pi} & \text{for } m \text{ odd} \\
  0 & \text{for } m \text{ even}
\end{cases}$$

Let's plot the graph as a function of T

In [298]:
T = np.linspace(-3/2, 3/2, 1000)

def x(m, t):
    x = 0
    for n in range(1, m+1):
        b = 1/(n*pi) * (1 - (-1)**n)
        s = sin(n*pi*t)
        x += 2*b*s 
    return x 

In [299]:
M = [1, 2, 5, 10, 100]

plt.figure(figsize = (10, 4))
for i in range(len(M)):
    plt.plot(T, x(M[i], T), label=M[i])

plt.plot(T, square(pi*T), label='Exact')
plt.title('Fourier series approximation of an odd square-wave function for $n$ terms and the exact function')
plt.xlabel('$T$')
plt.legend()

<Figure size 720x288 with 1 Axes>

1
